### Simple On/Off Keying ATPG implementation

 - InjectionPoints: PullUp/PullDown for each node (automatically generated)
 - Observers: Chosen by the user
 - Faults: 6 fault model for each transistor; open/short for each resistor; short for each capacitor (automatically generated)

In [ ]:
import os

from spicetestlib.base import NetList
from spicetestlib.simulator import LTSpiceSimulator
from spicetestlib.test_utilities import InjectionPoint, InverterObserver
from spicetestlib.faults import resistor_fault_factory, capacitor_fault_factory, fet_fault_factory

NETLIST_PATH = os.path.join("SPICE",'amplifier_base.net')
SPICE_SIM_DATA_PATH = "SPICE_SIM_DATA"

SUPPLY_NODE = 'VDD'
GROUND_NODE = '0'
VDD = 3.3
VSS = 0

netlist = NetList(NETLIST_PATH)

# create all possible injection points
injection_points = []
for node in netlist.net.get_all_nodes():

    # skip supply and ground nodes
    if node.lower() in [SUPPLY_NODE.lower(), GROUND_NODE.lower()]:
        continue

    # create both pull up and pull down
    injection_points.append(InjectionPoint(netlist, node, False))
    injection_points.append(InjectionPoint(netlist, node, True))

observation_points = [
    InverterObserver(netlist, 'OUT'),
    InverterObserver(netlist, 'DIF+'),
]

faults = []
for c in netlist.net.get_components():
    if c.startswith('R'):
        faults.extend(resistor_fault_factory(netlist, c))
    elif c.startswith('C'):
        faults.extend(capacitor_fault_factory(netlist, c))
    elif c.startswith('M'):
        faults.extend(fet_fault_factory(netlist, c))

# add a transient analysis
netlist.net.add_instruction(".tran 200n")

# insert and activate all observation points
for obs in observation_points:
    obs.inject()
    obs.activate()
    print(str(obs))

# insert all injection points
for inj in injection_points:
    inj.inject()
    print(str(inj))

In [ ]:
sim = LTSpiceSimulator(output_folder=SPICE_SIM_DATA_PATH)

# run simulation without injection points to acquire faults detected by observation points without injection
# set expected values
log, raw, raw_op = sim.run_now(netlist, f'expected.net')
for obs in observation_points:
    obs.observe_expected(log, raw, raw_op)

results_non_inj_bool = [[] for obs in observation_points]
results_non_inj_float = [[] for obs in observation_points]

faults_detected = []
faults_left = []

# iterate over all faults
for fault in faults:
    fault.inject()
    print(f'Running simulation for fault: {fault}')
    result = sim.run_now_n_eval(netlist, f'fault.net', observation_points)
    for i, obs in enumerate(observation_points):
        results_non_inj_bool[i].append(result[0][i])
        results_non_inj_float[i].append(result[1][i][0])
    if any([not res for res in result[0]]):
        faults_detected.append(fault)
    else:
        faults_left.append(fault)
    fault.eject()

In [ ]:
results_bool = []
results_float = []

for i, inj in enumerate(injection_points):
    
    inj.activate()
    
    print(f"Running simulation for {inj}")

    results_bool.append([])
    results_float.append([])

    # set expected values
    log, raw, raw_op = sim.run_now(netlist, f'expected.net')
    for obs in observation_points:
        obs.observe_expected(log, raw, raw_op)
    
    for j, fault in enumerate(faults):
        fault.inject()
        print(f"  Running simulation for {fault}")
        result = sim.run_now_n_eval(netlist, f'sim.net', observation_points)
        results_bool[i].append(result[0])
        results_float[i].append([r[0] for r in result[1]])
        fault.eject()

    inj.deactivate()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# plotting of results
pixels_out = np.zeros(shape = (len(injection_points), len(faults)))
pixels_stage1 = np.zeros(shape = (len(injection_points), len(faults)))
for i in range(len(injection_points)):
    for j in range(len(faults)):
        # output observer
        if not results_bool[i][j][0]:
            pixels_out[i,j] = 1
        elif results_float[i][j][0] != 0 and abs(results_float[i][j][0]) > 1e-11:
            pixels_out[i,j] = 0.5
        # stage 1 observer
        if not results_bool[i][j][1]:
            pixels_stage1[i,j] = 1
        elif results_float[i][j][1] != 0 and abs(results_float[i][j][0]) > 1e-11:
            pixels_stage1[i,j] = 0.5

plt.figure(figsize=(5, 3.5))
fig, ax = plt.subplots(2, 1)
ax[0].imshow(pixels_out, cmap='Greens', aspect='auto', extent=[0.5, len(faults)+0.5, len(injection_points)+0.5, 0.5])
ax[0].set_title('Output Observer')
ax[1].imshow(pixels_stage1, cmap='Greens', aspect='auto', extent=[0.5, len(faults)+0.5, len(injection_points)+0.5, 0.5])
ax[1].set_title('Stage 1 Observer')
for axis in ax:
    axis.set_xlabel('Fault')
    axis.set_ylabel('Injection Point')
    axis.set_xticks(np.arange(0.5, len(faults), 1), minor=True)
    axis.set_yticks(np.arange(0.5, len(injection_points), 1), minor=True)
    axis.grid(which='minor', color='black', linestyle='-', linewidth=0.5)

# add legend
legend_patches = [mpatches.Patch(color='#f7fcf5', label='Not Detected'),
                  mpatches.Patch(color='#73c375', label='Detectable over Timing'),
                  mpatches.Patch(color='#00441b', label='Detected')]

fig.legend(handles=legend_patches, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=3)

plt.tight_layout()
plt.savefig(os.path.join('figures', 'structural_overall.pdf'), format='pdf', bbox_inches='tight', pad_inches=0)
plt.show()
# plot only the first dataset

plt.figure(figsize=(5, 3.5))
plt.imshow(pixels_out, cmap='Greens', aspect='auto', extent=[0.5, len(faults)+0.5, len(injection_points)+0.5, 0.5])
plt.xlabel('Fault')
plt.ylabel('Injection Point')
plt.xticks([1, *np.arange(5, len(faults)+1, 5)])
plt.xticks(np.arange(0.5, len(faults), 1), minor=True)
plt.yticks([1, *np.arange(4, len(injection_points)+1, 2)])
plt.yticks(np.arange(0.5, len(injection_points), 1), minor=True)
plt.grid(which='minor', color='black', linestyle='-', linewidth=0.5)

# add legend
legend_patches = [mpatches.Patch(color='#00441b', label='Detectable'),
                  mpatches.Patch(color='#73c375', label='Detectable over Timing'),
                  mpatches.Patch(color='#f7fcf5', label='Undetectable'),]

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.legend(handles=legend_patches, loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2)

plt.savefig(os.path.join('figures', 'structural_out.pdf'), format='pdf', bbox_inches='tight', pad_inches=0)
plt.title('Output Observer')
plt.show()

In [ ]:
# plot timing for injection 8

# gather timing difference data
timing_diff = []
timing_irrelev = []

for j in range(len(faults)):
    if not results_bool[8][j][0] or results_float[8][j][0] < -1:
        timing_irrelev.append(j)
        timing_diff.append(1e-6)
    else:
        timing_diff.append(np.abs(results_float[8][j][0]))

plt.figure(figsize=(5, 3.5))

plt.bar(range(len(timing_diff)), timing_diff, color='blue')
plt.xlabel('Fault')
plt.ylabel('Absolute Timing Difference [s]')
plt.xlim(-0.5, len(timing_diff)-0.5)
plt.yscale('log')

# add coloring and legend
for i in range(len(timing_irrelev)):
    plt.bar(timing_irrelev[i], timing_diff[timing_irrelev[i]], color='green')
legend_patches = [mpatches.Patch(color='blue', label='Detectable over Timing'),
                  mpatches.Patch(color='green', label='Detectable without Timing')]
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.legend(handles=legend_patches, loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2)
plt.savefig(os.path.join('figures', 'structural_pull_down_dif+.pdf'), format='pdf', bbox_inches='tight', pad_inches=0)
plt.title('Timing Difference for Pull Down Injection @ DIF+')
plt.show()

In [ ]:
# plot results without injection

pixels_out = np.zeros(shape = (len(faults)))
pixels_stage1 = np.zeros(shape = (len(faults)))

for j in range(len(faults)):
    # output observer
    if not results_non_inj_bool[0][j]:
        pixels_out[j] = 1
    # stage 1 observer
    if not results_non_inj_bool[1][j]:
        pixels_stage1[j] = 1

fig, ax = plt.subplots(2, 1, figsize=(5, 3.5))
ax[0].bar(range(1,len(faults)+1), pixels_out, color='green')
ax[0].set_title('Output Observer')
ax[1].bar(range(1,len(faults)+1), pixels_stage1, color='green')
ax[1].set_title('Stage 1 Observer')
for axis in ax:
    axis.set_xlim(0.5, len(faults)+0.5)
    axis.set_xlabel('Fault')
    axis.set_ylabel('Detected')
    axis.set_xticks([1, *np.arange(5, len(faults)+1, 5)])
    axis.set_xticks(np.arange(0.5, len(faults)+0.5, 1), minor=True)
    axis.set_yticks([0, 1])
    axis.set_ylim([0,1])
    axis.grid(which='minor', color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.savefig(os.path.join('figures', 'structural_overall_non_inj.pdf'), format='pdf', bbox_inches='tight', pad_inches=0)
plt.show()
